# L2D Showcase

The following code is a modified version of [test_learned_on_benchmark.py](l2d/test_learned_on_benchmark.py)

In [1]:
import sys
sys.path.append("repo")
import os
os.chdir("repo")

In [2]:
from mb_agg import *
from agent_utils import *
from Params import configs
from JSSP_Env import SJSSP
from PPO_jssp_multiInstances import PPO

import time
import torch
import argparse
import numpy as np
import itertools

In [3]:
def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

In [50]:
def solve_instance(instance: str, model: str, plan=None, device='cpu', machine_start_times=None, t:int=0):
    '''Solves instance using given model and prints out the makespan
    
      Args:
          instance - file with the instance in Taillard specification
          model - network model to use as an agent

      Returns:
          makespan of the solution,
          dispatch times of operations,
          actions executed by the agent
    '''
    # parse the instance
    jobs, machines, times, orders = parse_instance_taillard(instance)

    # load agents environment
    env = SJSSP(n_j=jobs, n_m=machines)
    adj, fea, candidate, mask = env.reset((times, orders))
    ep_reward = - env.max_endTime
    if machine_start_times is not None:
        env.mchsStartTimes = machine_start_times

    # load the agent
    ppo = PPO(configs.lr, configs.gamma, configs.k_epochs, configs.eps_clip,
              n_j=jobs,
              n_m=machines,
              num_layers=configs.num_layers,
              neighbor_pooling_type=configs.neighbor_pooling_type,
              input_dim=configs.input_dim,
              hidden_dim=configs.hidden_dim,
              num_mlp_layers_feature_extract=configs.num_mlp_layers_feature_extract,
              num_mlp_layers_actor=configs.num_mlp_layers_actor,
              hidden_dim_actor=configs.hidden_dim_actor,
              num_mlp_layers_critic=configs.num_mlp_layers_critic,
              hidden_dim_critic=configs.hidden_dim_critic)
    ppo.policy.load_state_dict(torch.load(model, map_location=torch.device('cpu')))
    g_pool_step = g_pool_cal(graph_pool_type=configs.graph_pool_type,
                             batch_size=torch.Size([1, env.number_of_tasks, env.number_of_tasks]),
                             n_nodes=env.number_of_tasks,
                             device=device)

    # run the experiment
    for i in itertools.count():
        fea_tensor = torch.from_numpy(np.copy(fea)).to(device)
        adj_tensor = torch.from_numpy(np.copy(adj)).to(device).to_sparse()
        candidate_tensor = torch.from_numpy(np.copy(candidate)).to(device)
        mask_tensor = torch.from_numpy(np.copy(mask)).to(device)

        action = None
        # choose action from partial plan if given
        if plan is not None and i < len(plan):
            env.set_current_time(plan[i][1])
            action = np.int64(plan[i][0])
            assert action in candidate, f"Action {action} not in candidate {candidate}" 

        # if no action was chosen from partial plan, use agent to choose
        if action is None:
            env.set_current_time(t)
            with torch.no_grad():
                pi, _ = ppo.policy(x=fea_tensor,
                                   graph_pool=g_pool_step,
                                   padded_nei=None,
                                   adj=adj_tensor,
                                   candidate=candidate_tensor.unsqueeze(0),
                                   mask=mask_tensor.unsqueeze(0))
                action = greedy_select_action(pi, candidate)


        adj, fea, reward, done, candidate, mask = env.step(action)
        ep_reward += reward

        if done:
            break

    start_times = env.LBs - times
    return env.posRewards - ep_reward, start_times, env.partial_sol_sequeence


# 30x20 Taillard's instances
I test the "30x20" model on Taillard's 30x20 instances.

In [5]:
MODEL = "SavedNetwork/30_20_1_99.pth"
BENCHMARKS_PATH = "../../../benchmarks/jssp/ta_instances/Taillard_specification/"
INSTANCES = [
    'ta41.txt',
    'ta42.txt',
    'ta43.txt',
    'ta44.txt',
    'ta45.txt',
    'ta46.txt',
    'ta47.txt',
    'ta48.txt',
    'ta49.txt',
    'ta50.txt',
]

for instance in INSTANCES:
    makespan, _, _ = solve_instance(BENCHMARKS_PATH + instance, MODEL)
    print(f"Makespan of instance '{instance}': {makespan}")

/Users/marosbratko/Graph-neural-networks-and-deep-reinforcement-learning-in-job-scheduling/models/L2D/repo/mb_agg.py:44: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:607.)
  graph_pool = torch.sparse.FloatTensor(idx, elem,


Makespan of instance 'ta41.txt': 2592.0
Makespan of instance 'ta42.txt': 2715.0
Makespan of instance 'ta43.txt': 2431.0
Makespan of instance 'ta44.txt': 2712.0
Makespan of instance 'ta45.txt': 2651.0
Makespan of instance 'ta46.txt': 2852.0
Makespan of instance 'ta47.txt': 2502.0
Makespan of instance 'ta48.txt': 2525.0
Makespan of instance 'ta49.txt': 2497.0
Makespan of instance 'ta50.txt': 2507.0


# All benchmarks

I test the "30x20" model on all available instances in Taillard specification.

In [6]:
def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/jssp/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [7]:
MODEL = "SavedNetwork/30_20_1_99.pth"
instances = get_all_instances_in_taillard_specification()
for instance in instances:
    makespan, _, _ = solve_instance(instance, MODEL)
    print(f"Makespan of instance '{instance.split('/')[-1]}': {makespan}")

Makespan of instance 'dmu36.txt': 7308.0
Makespan of instance 'dmu22.txt': 5606.0
Makespan of instance 'dmu23.txt': 5267.0
Makespan of instance 'dmu37.txt': 7364.0
Makespan of instance 'dmu21.txt': 5706.0
Makespan of instance 'dmu35.txt': 6457.0
Makespan of instance 'dmu09.txt': 4377.0
Makespan of instance 'dmu08.txt': 4071.0
Makespan of instance 'dmu34.txt': 6514.0
Makespan of instance 'dmu20.txt': 4715.0
Makespan of instance 'dmu18.txt': 4908.0
Makespan of instance 'dmu24.txt': 5626.0
Makespan of instance 'dmu30.txt': 6130.0
Makespan of instance 'dmu31.txt': 6971.0
Makespan of instance 'dmu25.txt': 4945.0
Makespan of instance 'dmu19.txt': 4862.0
Makespan of instance 'dmu33.txt': 6155.0
Makespan of instance 'dmu27.txt': 6140.0
Makespan of instance 'dmu26.txt': 6089.0
Makespan of instance 'dmu32.txt': 6572.0
Makespan of instance 'dmu55.txt': 6582.0
Makespan of instance 'dmu41.txt': 4738.0
Makespan of instance 'dmu69.txt': 8729.0
Makespan of instance 'dmu68.txt': 8713.0
Makespan of inst

KeyboardInterrupt: 

# Dynamic JSSP

In dynamic JSSP, only a subset of jobs is known at the beginning. The rest of jobs arrives dynamically online.

The following attempt to expand L2D to being dynamic is inspired by paper [Large-scale Dynamic Scheduling for Flexible Job-shop with Random Arrivals of New Jobs by Hierarchical Reinforcement Learning](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10114974), where authors schedule newly incoming jobs and reschedule not yet executed operations, already executed operations can not be rescheduled. During each rescheduling, they formulate static FJSP and solve it. They use cache for incoming jobs and an agent choosing either to add jobs from cache to scheduling problem, or keep them in cache. I will skip this agent and always add new jobs to scheduling problem.

Similarly to the paper, I will model the arrival of new jobs as poisson process with average arrival time following an exponential distribution.

In [92]:
from datetime import datetime

def get_dynamics_jssp(instance):
    '''Turns static JSSP instance to dynamic

      Args:
        filename of static JSSP instance

      Returns:
        list of jobs known at the beginning
        dictionary of arriving jobs as  as {time_of_arrival: (operations, machines)} 
    '''
    J, M, processor_times, orders_of_machines = parse_instance_taillard(instance)

    indices = np.arange(J)
    np.random.shuffle(indices)

    # separate jobs into known jobs and arriving jobs
    jobs_known_at_the_beginning = [(processor_times[i], orders_of_machines[i]) for i in indices[J//2:]]
    arriving_jobs_indeces = indices[:J//2]

    # calculate beta = 1/lambda
    average_time_between_arrivals = processor_times.mean()
    
    t = 0
    arriving_jobs = {}
    for index in arriving_jobs_indeces:
        t += int(np.random.exponential(scale=average_time_between_arrivals)) + 1
        arriving_jobs[t] = (processor_times[index], orders_of_machines[index])

    return jobs_known_at_the_beginning, arriving_jobs

def save_static_jssp_taillard(jobs):
    '''Saves list of jobs as static JSSP instance in taillards specification
        
      Args:
        list of jobs to save

      Returns:
        filename where JSSP instance was saved to
    '''
    J, M = len(jobs), len(jobs[0][0])
    formatted_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    with open(f"/tmp/{J}_{M}_{formatted_datetime}.txt", 'w') as f:
        f.write(f"{J} {M}\n")
        for job in jobs:
            times, _ = job
            f.write(" ".join(map(str, times)) + '\n')
        for job in jobs:
            _, orders = job
            f.write(" ".join(map(str, orders)) + '\n')  

    return f"/tmp/{J}_{M}_{formatted_datetime}.txt"

In [93]:
def solve_dynamic_jssp(instance, model):
    '''Turns static JSSP in Taillard specification instance to dynamic and solves it

      Args: 
        instance to solve

      Returns: 
        makespan
    '''
    # turn static JSSP instance to dynamic
    known_jobs, arriving_jobs = get_dynamics_jssp(instance)
    # print(f"instance={instance.split('/')[-1]}, known_jobs={len(known_jobs)}, arriving_jobs={len(arriving_jobs)}")
    latest_time_of_arrival = max(arriving_jobs)
    # print(f"Latest job arrives at {latest_time_of_arrival} and has total makespan {arriving_jobs[latest_time_of_arrival][0].max()}")
    
    # solve static JSSP with jobs known initially
    makespan, start_times, actions = solve_instance(save_static_jssp_taillard(known_jobs), model)
    t = 0
    plan = []
    while True:
        t += 1
        
        # no jobs left
        if not arriving_jobs:
            break
    
        # no job arrived
        if not t in arriving_jobs:
            continue
    
        # new job arrived, remove not yet executed operations from the plan
        J, M = len(known_jobs), len(known_jobs[0][0])
        for i in range(len(actions)):
            row = actions[i] // M
            col = actions[i] % M

            if not start_times[row][col] < t:
                continue
                
            if (actions[i], start_times[row][col]) in plan:
                continue
    
            plan.append((actions[i], start_times[row][col]))

        # add new job to the plan, with times shifted to current time t
        new_job = arriving_jobs.pop(t)
        known_jobs.append(new_job)
        
        # create new schedule WHILE REUSING THE ALREADY EXECUTED PLAN
        makespan, start_times, actions = solve_instance(save_static_jssp_taillard(known_jobs), MODEL, plan=plan, t=t)

    return makespan, plan, start_times

## Run the dynamic experiment

Run the experiment on 10 taillard instances.

In [94]:
MODEL = "SavedNetwork/30_20_1_99.pth"
BENCHMARKS_PATH = "../../../benchmarks/jssp/ta_instances/Taillard_specification/"
INSTANCES = [
    'ta41.txt',
    'ta42.txt',
    'ta43.txt',
    'ta44.txt',
    'ta45.txt',
    'ta46.txt',
    'ta47.txt',
    'ta48.txt',
    'ta49.txt',
    'ta50.txt',
]

for instance in INSTANCES:
    static_makespan, _, _ = solve_instance(BENCHMARKS_PATH + instance, MODEL)
    dynamic_makespan, plan, start_times = solve_dynamic_jssp(BENCHMARKS_PATH + instance, MODEL)
    print(f"instance={instance}, static_makespan={static_makespan}, dynamic_makespan={dynamic_makespan}")

instance=ta41.txt, static_makespan=2592.0, dynamic_makespan=3153.0
instance=ta42.txt, static_makespan=2715.0, dynamic_makespan=2575.0
instance=ta43.txt, static_makespan=2431.0, dynamic_makespan=2669.0
instance=ta44.txt, static_makespan=2712.0, dynamic_makespan=2808.0
instance=ta45.txt, static_makespan=2651.0, dynamic_makespan=2859.0
instance=ta46.txt, static_makespan=2852.0, dynamic_makespan=2822.0
instance=ta47.txt, static_makespan=2502.0, dynamic_makespan=2861.0
instance=ta48.txt, static_makespan=2525.0, dynamic_makespan=2518.0
instance=ta49.txt, static_makespan=2497.0, dynamic_makespan=2859.0
instance=ta50.txt, static_makespan=2507.0, dynamic_makespan=2839.0
